![schema](schema.png)
 

In [1]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

#!pip install unidecode
import unidecode

# 1 - imports
from datetime import datetime, date


In [2]:
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, Boolean, Date, MetaData, ForeignKey
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

#from base import Base



## Scrapping des matchs et buteurs

In [ ]:
old_url="https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/1re-journee"
url="https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats"
page_response=requests.get(url,timeout=5)
soup =BeautifulSoup(page_response.content, 'html.parser')

In [ ]:
list_equipe_ext =[]
liste_score_ext = []
list_equipe_dom = []
liste_score_dom = []
lieu_match = []
scorer_names_dom= []
scorer_names_ext=[]
date_match=[]
match_scorer_dom=[]
match_scorer_ext=[]
num_journee=[]
vainqueur=[]

In [ ]:
players=pd.DataFrame()
journees=soup.find(class_="SelectNav__select").find_all("option")
i=1
for journee in journees:
    json_data=requests.get(journee['value'])
    data = json_data.json() 
    # correspond à une liste de match
    journee= data['items'] 
    for date in journee :
        try : 
            for match in date['items']:
                list_equipe_ext.append(match['event']['specifics']['exterieur']['equipe']['nom'])
                #print (match['event']['lien_web'])
                liste_score_ext.append(match['event']['specifics']['score']['exterieur'])
                list_equipe_dom.append(match['event']['specifics']['domicile']['equipe']['nom'])
                liste_score_dom.append((match['event']['specifics']['score']['domicile']))
                vainqueur.append(match['event']['specifics']['vainqueur'])
                lieu_match.append(match['event']['lieu']['ville'])        
                date_match.append(match['date'])
                num_journee.append(i)
#               players=pd.concat(players,table_equipes_match)
                url_match=match['event']['lien_web']
                print(url_match)
                match_detail=requests.get(url_match,timeout=5)
                soup_match_detail =BeautifulSoup(match_detail.content, 'html.parser')
                match_detail=soup_match_detail.find_all(class_="Scoreboard__goals")
                match_scorer_ext=[]
                match_scorer_dom=[]
                for scorer in match_detail :
                    span=scorer.find_all('span', recursive=False)
                    list_scorers=scorer.text.split() 
                    try : 
                        if int(list_scorers[0][0:2]): 
                            if len(span[2]) != 0:
                                match_scorer_ext.append(span[1].text + span[2].text)
                            else : 
                                match_scorer_ext.append(span[1].text)
                    except :
                            if len(span[0]) != 0:
                                match_scorer_dom.append(span[1].text+ span[0].text)
                            else:
                                match_scorer_dom.append(span[1].text)
                scorer_names_ext.append(match_scorer_ext)
                scorer_names_dom.append(match_scorer_dom)
        except KeyError:
            continue
    i+=1
        
d_ext = {'Exterior_team': list_equipe_ext , 'score_exterior': liste_score_ext,'scorer_names_ext':scorer_names_ext,
         'lieu_match':lieu_match, 'date':date_match,'journee':num_journee, 'vainqueur': vainqueur}
d_dom ={'Domicile_team': list_equipe_dom ,'score_dom': liste_score_dom, 'scorer_names_dom':scorer_names_dom,
     'lieu_match':lieu_match, 'date':date_match,'journee':num_journee, 'vainqueur': vainqueur}
d_matchs= {'lieu_match':lieu_match, 'date':date_match,'journee':num_journee}
df_goals_ext = pd.DataFrame(data=d_ext)
df_goals_dom=pd.DataFrame(data=d_dom)
df_matches= pd.DataFrame(data=d_matchs)
#d_dom ={'Exterior_team': list_equipe_ext , 'score_exterior': liste_score_ext,'scorer_names_ext':scorer_names_ext,
#         'Domicile_team': list_equipe_dom ,'score_dom': liste_score_dom, 'scorer_names_dom':scorer_names_dom,
#     'lieu_match':lieu_match, 'date':date_match,'journee':num_journee}

In [ ]:
df_goals_ext.to_csv("goal_extGT.csv",index=False)
df_goals_dom.to_csv("goals_domGT.csv",index=False)
df_matches.to_csv("matches.csv",index=False)

In [ ]:
df_goals_dom

In [ ]:
df_goals_ext.head(15)

In [ ]:
df_matches.head()

## Ajout Météo


### mappping des régions

In [ ]:
set_villes=set(df_matches['lieu_match'])
set_villes

In [ ]:
mapping_region_ville={'Angers' : 'pays-de-la-loire',
                     'Bordeaux': 'aquitaine',
                     'Brest': 'bretagne',
                     'Dijon': 'bourgogne',
                     'Lens':'nord-pas-de-calais',
                     'Lille': 'nord-pas-de-calais',
                     'Lorient': 'bretagne',
                     'Lyon': 'rh-ne-alpes',
                     'Marseille': 'provence-alpes-c-te-d-azur',
                     'Metz': 'lorraine',
                     'Monaco':'provence-alpes-c-te-d-azur',
                     'Montpellier': 'languedoc-roussillon',
                     'Nantes' : 'pays-de-la-loire',
                     'Nice': 'provence-alpes-c-te-d-azur',
                     'Nîmes': 'languedoc-roussillon',
                     'Paris' :'ile-de-france',
                     'Reims' : 'champagne-ardenne',
                     'Rennes': 'bretagne',
                     'Saint-Étienne': 'rh-ne-alpes',
                     'Strasbourg' : 'alsace'}

In [ ]:
regions=[]
[regions.append(mapping_region_ville[lieu])for lieu in df_matches['lieu_match']]
df_matches['region']=regions   

### trnsofrmation date dans le bon format

In [ ]:
df_matches['date']=[value.split('+')[0] for value in df_matches['date']]

In [ ]:
df_matches['date']=[datetime.strptime(value,"%Y-%m-%dT%H:%M:%S").date() for value in df_matches['date']]

In [ ]:
#df_matches['date']=date_clean

In [ ]:
#year_month_day=[]
#[year_month_day.append(row.date())for row in df_matches['date']]
#df_matches['date']=year_month_day

In [ ]:
df_matches['date']=[str(x).replace('-','/') for x in df_matches['date']]

### Creation liste urls

In [ ]:
meteo_url_base="https://www.historique-meteo.net/france/"

In [ ]:
urls_meteo=[]
for index,row in df_matches[['region','lieu_match','date']].iterrows(): 
    urls_meteo.append((meteo_url_base+row['region']+'/'+ unidecode.unidecode(row['lieu_match'].lower())+'/'+row['date']+'/'))

In [ ]:
urls_meteo

### Scrapping

In [ ]:
temp_max=[]
temp_min=[]
precipitation=[]
for url in urls_meteo:
    print(url)
    meteo_match=requests.get(url,timeout=5)
    soup_meteo_match =BeautifulSoup(meteo_match.content, 'html.parser')
    temp_max.append(soup_meteo_match.find("td", text="Température maximale").find_next_sibling("td").next_sibling.next_sibling.text)
    temp_min.append(soup_meteo_match.find("td", text="Température minimale").find_next_sibling("td").next_sibling.next_sibling.text)
    try : 
        precipitation.append(soup_meteo_match.find("td", text="Précipitations").find_next_sibling("td").next_sibling.next_sibling.text)
    except:
        precipitation.append('0mm')
df_matches['temp_max']=temp_max
df_matches['temp_min']=temp_min
df_matches['precipitations']=precipitation

In [ ]:
df_matches.tail()

In [ ]:
df_matches.to_csv('matches_meteo.csv')

## SQL Alchemy ORM

In [113]:
engine = create_engine('sqlite:///football_orm.sqlite', echo = True)
Session = sessionmaker(bind=engine)

Base = declarative_base()

### Creation database

In [114]:
class Teams(Base):
    __tablename__ = 'teams'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    city = Column(String)
    coach_name = Column(String)
    players=relationship("Players", back_populates="team")
    matches= relationship("MatchsTeamsLink", back_populates="teams")

    
    def __init__(self, name, city, coach_name):
        self.name = name
        self.city = city
        self.coach_name = coach_name
#        self.rank= rank

class Players(Base):
    __tablename__ = 'players'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    position = Column(String)
    birthdate = Column(Date)
    nationality = Column(String)
# 1 player has only one club to facilitate the exercice
    team_id = Column(Integer, ForeignKey('teams.id'))
    team = relationship("Teams", back_populates="players")
    matchs= relationship("Goals", back_populates="player")
    
    def __init__(self, name, position, birthdate, nationality,team_id):
        self.name = name
        self.position = position
        self.birthdate = birthdate
        self.nationality= nationality
        self.team_id= team_id
        
class Matches(Base):
    __tablename__ = 'matches'

    id = Column(Integer, primary_key=True)
    date = Column(Date)
    place = Column(String)
    rainfall = Column(Float)
    temperature = Column(Float)
    teams = relationship("MatchsTeamsLink",back_populates="matches")
    players=relationship("Goals",back_populates="match")

       
    def __init__(self, date, place, rainfall, temperature):
        self.date = date
        self.place = place
        self.rainfall = rainfall
        self.temperature= temperature
        
        
class MatchsTeamsLink(Base):
    __tablename__ = 'matches_teams'
    id = Column(Integer , primary_key= True)
    
    team_id = Column(Integer, ForeignKey('teams.id'))
    matchs_id = Column(Integer, ForeignKey('matches.id'))

    home = Column(Boolean)
    points= Column(Integer)
    team_goals=Column(Integer)
    teams = relationship("Teams", back_populates="matches")
    matches = relationship("Matches", back_populates="teams")
    
    def __init__(self, team_id, matchs_id, home,points,team_goals):
        self.team_id = team_id
        self.matchs_id = matchs_id
        self.home = home
        self.points = points
        self.team_goals = team_goals
        
class Goals(Base):
    __tablename__ = 'goals'
    
    id = Column(Integer , primary_key= True)
    player_id = Column(Integer, ForeignKey('players.id'))
    match_id = Column(Integer, ForeignKey('matches.id'))
    
    match = relationship("Matches", back_populates='players')
    player = relationship("Players", back_populates='matchs')
   
    goal_type = Column(String)

    
    def __init__(self, player_id, match_id, goal_type):
        self.player_id = player_id
        self.match_id = match_id
        self.goal_type = goal_type

In [115]:
# 2 - generate database schema
Base.metadata.create_all(engine)

# 3 - create a new session
session = Session()

2021-07-01 13:38:26,835 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:38:26,839 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("teams")
2021-07-01 13:38:26,841 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 13:38:26,847 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("teams")
2021-07-01 13:38:26,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 13:38:26,855 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("players")
2021-07-01 13:38:26,857 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 13:38:26,862 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("players")
2021-07-01 13:38:26,865 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 13:38:26,869 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("matches")
2021-07-01 13:38:26,871 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-01 13:38:26,877 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("matches")
2021-07-01 13:38:26,878 INFO sqlalchemy.engine.Engine [raw sql] ()
202

### Teams Table

In [116]:
df_teams=pd.read_csv('club_entraineurs.csv', index_col=0)

In [117]:
df_teams

,Ville du Club,Nom du Club,Entraineur
0,Angers,Angers Sporting Club de l'Ouest,Gérald BATICLE
1,Bordeaux,Football Club des Girondins de Bordeaux,Jean-Louis GASSET
2,Brest,Stade Brestois 29,Michel DER ZAKARIAN
3,Dijon,Dijon Football Côte-d'Or,David LINARES
4,Lens,Racing Club de Lens,Franck HAISE
5,Lille,LOSC Lille,Christophe GALTIER
6,Lorient,Football Club Lorient-Bretagne Sud,Christophe PELISSIER
7,Lyon,Olympique Lyonnais,Peter BOSZ
8,Marseille,Olympique de Marseille,Jorge SAMPAOLI
9,Metz,Football Club de Metz,Frédéric ANTONETTI


In [118]:
# adding values in teams table
for i,team in df_teams[["Nom du Club","Ville du Club","Entraineur"]].iterrows() :
    new_team = Teams(team.values[0],team.values[1],team.values[2])
    session.add(new_team)

In [119]:
session.commit()
session.close()

2021-07-01 13:38:27,582 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:38:27,590 INFO sqlalchemy.engine.Engine INSERT INTO teams (name, city, coach_name) VALUES (?, ?, ?)
2021-07-01 13:38:27,592 INFO sqlalchemy.engine.Engine [generated in 0.00213s] ("Angers Sporting Club de l'Ouest", 'Angers', 'Gérald BATICLE')
2021-07-01 13:38:27,598 INFO sqlalchemy.engine.Engine INSERT INTO teams (name, city, coach_name) VALUES (?, ?, ?)
2021-07-01 13:38:27,601 INFO sqlalchemy.engine.Engine [cached since 0.01129s ago] ('Football Club des Girondins de Bordeaux', 'Bordeaux', 'Jean-Louis GASSET')
2021-07-01 13:38:27,604 INFO sqlalchemy.engine.Engine INSERT INTO teams (name, city, coach_name) VALUES (?, ?, ?)
2021-07-01 13:38:27,605 INFO sqlalchemy.engine.Engine [cached since 0.01626s ago] ('Stade Brestois 29', 'Brest', 'Michel DER ZAKARIAN')
2021-07-01 13:38:27,608 INFO sqlalchemy.engine.Engine INSERT INTO teams (name, city, coach_name) VALUES (?, ?, ?)
2021-07-01 13:38:27,610 INFO sqlalch

### Players table

In [120]:
# adding players in players table 
df_players= pd.read_csv('team_players_final.csv', index_col=0)  

In [121]:
df_players=df_players.rename(columns={"Unnamed: 0.1":"shirt_num", df_players.columns[3]: "Birthdate"})

In [122]:
df_players=df_players.drop(['id','num_id'],axis=1)

In [123]:
df_players=df_players.reset_index(drop=True)

In [124]:
teams_id = {}
result = session.query(Teams).all()
for team_obj in result:
    teams_id[team_obj.city] = team_obj.id

2021-07-01 13:38:28,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:38:28,097 INFO sqlalchemy.engine.Engine SELECT teams.id AS teams_id, teams.name AS teams_name, teams.city AS teams_city, teams.coach_name AS teams_coach_name 
FROM teams
2021-07-01 13:38:28,101 INFO sqlalchemy.engine.Engine [generated in 0.00361s] ()


In [125]:
i=0
for value in df_players['club']:
    if value.split()[0] in teams_id.keys():
        df_players.loc[i,'team_id']=teams_id[value.split()[0]]
    i+=1

In [126]:
df_players['Birthdate']=df_players['Birthdate'].fillna('01/01/00')

In [127]:
# adding values in players table
for i,player in df_players[["Nom","Pos.","Birthdate", "Pays", "club", "team_id"]].iterrows() :
    new_player = Players(player.values[0],player.values[1],datetime.strptime(str(player.values[2]), '%d/%m/%y').date(),player.values[3],player.values[5])
    session.add(new_player)

In [128]:
session.commit()
session.close()

2021-07-01 13:38:29,469 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,471 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ('P. Bernardoni', 'Gar.', '1997-04-18', 'FRA', 1.0)
2021-07-01 13:38:29,476 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,478 INFO sqlalchemy.engine.Engine [cached since 0.009659s ago] ('L. Butelle', 'Gar.', '1983-04-03', 'FRA', 1.0)
2021-07-01 13:38:29,481 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,484 INFO sqlalchemy.engine.Engine [cached since 0.01576s ago] ('D. Petkovic', 'Gar.', '1993-05-25', 'MNE', 1.0)
2021-07-01 13:38:29,487 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:2

2021-07-01 13:38:29,648 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,650 INFO sqlalchemy.engine.Engine [cached since 0.1815s ago] ('Y. Fortuné', 'Att.', '1999-01-30', 'FRA', 1.0)
2021-07-01 13:38:29,653 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,655 INFO sqlalchemy.engine.Engine [cached since 0.1863s ago] ('S. Thioub', 'Att.', '1995-06-01', 'SEN', 1.0)
2021-07-01 13:38:29,657 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,659 INFO sqlalchemy.engine.Engine [cached since 0.1911s ago] ('B. Costil', 'Gar.', '1987-07-03', 'FRA', 2.0)
2021-07-01 13:38:29,662 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,664 

2021-07-01 13:38:29,852 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,856 INFO sqlalchemy.engine.Engine [cached since 0.3878s ago] ('I. Diarra', 'Att.', '1998-05-19', 'FRA', 2.0)
2021-07-01 13:38:29,859 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,863 INFO sqlalchemy.engine.Engine [cached since 0.3944s ago] ('U. Hwang', 'Att.', '1992-08-28', 'CDS', 2.0)
2021-07-01 13:38:29,866 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,868 INFO sqlalchemy.engine.Engine [cached since 0.4s ago] ('S. Kalu', 'Att.', '1997-08-26', 'NGA', 2.0)
2021-07-01 13:38:29,871 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:29,875 INFO sq

2021-07-01 13:38:30,090 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,092 INFO sqlalchemy.engine.Engine [cached since 0.6243s ago] ('I. Cardona', 'Att.', '1997-08-08', 'FRA', 3.0)
2021-07-01 13:38:30,095 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,097 INFO sqlalchemy.engine.Engine [cached since 0.6291s ago] ('G. Charbonnier', 'Att.', '1988-12-27', 'FRA', 3.0)
2021-07-01 13:38:30,103 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,105 INFO sqlalchemy.engine.Engine [cached since 0.6371s ago] ('F. Honorat', 'Att.', '1996-08-11', 'FRA', 3.0)
2021-07-01 13:38:30,108 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:3

2021-07-01 13:38:30,315 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,319 INFO sqlalchemy.engine.Engine [cached since 0.8506s ago] ('A. Zagré', 'Mil.', '2001-10-04', 'FRA', 4.0)
2021-07-01 13:38:30,322 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,324 INFO sqlalchemy.engine.Engine [cached since 0.8555s ago] ('M. Baldé', 'Att.', '1995-11-06', 'GUB', 4.0)
2021-07-01 13:38:30,326 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,338 INFO sqlalchemy.engine.Engine [cached since 0.8706s ago] ('E. Belhadji', 'Att.', '2001-06-23', 'FRA', 4.0)
2021-07-01 13:38:30,342 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:30,344 I

...

2021-07-01 13:38:33,306 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,308 INFO sqlalchemy.engine.Engine [cached since 3.84s ago] ('M. Kapo', 'Mil.', '2001-01-19', 'FRA', 16.0)
2021-07-01 13:38:33,311 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,313 INFO sqlalchemy.engine.Engine [cached since 3.845s ago] ('E. Michut', 'Mil.', '2003-03-04', 'FRA', 16.0)
2021-07-01 13:38:33,317 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,320 INFO sqlalchemy.engine.Engine [cached since 3.851s ago] ('L. Paredes', 'Mil.', '1994-06-29', 'ARG', 16.0)
2021-07-01 13:38:33,323 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,324 INF

2021-07-01 13:38:33,487 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,492 INFO sqlalchemy.engine.Engine [cached since 4.024s ago] ('M. Cassama', 'Mil.', '1998-02-16', 'GUB', 17.0)
2021-07-01 13:38:33,494 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,497 INFO sqlalchemy.engine.Engine [cached since 4.028s ago] ('X. Chavalerin', 'Mil.', '1991-03-07', 'FRA', 17.0)
2021-07-01 13:38:33,500 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,502 INFO sqlalchemy.engine.Engine [cached since 4.034s ago] ('M. Doumbia', 'Mil.', '1994-08-15', 'MLI', 17.0)
2021-07-01 13:38:33,506 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33

2021-07-01 13:38:33,655 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,657 INFO sqlalchemy.engine.Engine [cached since 4.189s ago] ('C. Grenier', 'Mil.', '1991-01-07', 'FRA', 18.0)
2021-07-01 13:38:33,660 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,664 INFO sqlalchemy.engine.Engine [cached since 4.195s ago] ('J. Léa-Siliki', 'Mil.', '1996-06-12', 'FRA', 18.0)
2021-07-01 13:38:33,669 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,671 INFO sqlalchemy.engine.Engine [cached since 4.202s ago] ('J. Martin', 'Mil.', '1990-04-09', 'FRA', 18.0)
2021-07-01 13:38:33,674 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,

2021-07-01 13:38:33,862 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,864 INFO sqlalchemy.engine.Engine [cached since 4.396s ago] ('A. Aouchiche', 'Mil.', '2002-07-15', 'FRA', 19.0)
2021-07-01 13:38:33,868 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,870 INFO sqlalchemy.engine.Engine [cached since 4.402s ago] ('B. Benkhedim', 'Mil.', '2001-04-20', 'FRA', 19.0)
2021-07-01 13:38:33,872 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:33,874 INFO sqlalchemy.engine.Engine [cached since 4.406s ago] ('R. Boudebouz', 'Mil.', '1990-02-19', 'ALG', 19.0)
2021-07-01 13:38:33,878 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38

2021-07-01 13:38:34,145 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:34,152 INFO sqlalchemy.engine.Engine [cached since 4.684s ago] ('F. Guilbert', 'Déf.', '1994-12-24', 'FRA', 20.0)
2021-07-01 13:38:34,163 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:34,167 INFO sqlalchemy.engine.Engine [cached since 4.698s ago] ('L. Koné', 'Déf.', '1989-02-01', 'CIV', 20.0)
2021-07-01 13:38:34,177 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:34,185 INFO sqlalchemy.engine.Engine [cached since 4.717s ago] ('S. Mitrovic', 'Déf.', '1990-05-22', 'SER', 20.0)
2021-07-01 13:38:34,188 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:34,190

In [129]:
pd.read_sql_table('players', 'sqlite:///football_orm.sqlite') 

,id,name,position,birthdate,nationality,team_id
0,1,P. Bernardoni,Gar.,1997-04-18,FRA,1
1,2,L. Butelle,Gar.,1983-04-03,FRA,1
2,3,D. Petkovic,Gar.,1993-05-25,MNE,1
3,4,A. Bamba,Déf.,1990-04-25,CIV,1
4,5,K. Boma,Déf.,2002-11-20,FRA,1
...,...,...,...,...,...,...
619,620,L. Mothiba,Att.,1996-01-28,AFS,20
620,621,I. Saadi,Att.,1992-02-08,ALG,20
621,622,M. Sahi,Att.,2001-12-20,MLI,20
622,623,A. Waris,Att.,1991-09-19,GHA,20


### Matchs table

In [130]:
df_matches=pd.read_csv('matches_meteo.csv', index_col=0)

In [131]:
df_matches

,lieu_match,date,journee,region,temp_max,temp_min,precipitations
0,Bordeaux,2020/08/21,1,aquitaine,24°,21°,5mm
1,Dijon,2020/08/22,1,bourgogne,26°,18°,19mm
2,Lille,2020/08/22,1,nord-pas-de-calais,21°,17°,1mm
3,Monaco,2020/08/23,1,provence-alpes-c-te-d-azur,27°,24°,1mm
4,Lorient,2020/08/23,1,bretagne,19°,15°,1mm
...,...,...,...,...,...,...,...
375,Rennes,2021/05/23,38,bretagne,15°,5°,2mm
376,Nantes,2021/05/23,38,pays-de-la-loire,16°,8°,1mm
377,Saint-Étienne,2021/05/23,38,rh-ne-alpes,17°,7°,0mm
378,Reims,2021/05/23,38,champagne-ardenne,14°,7°,1mm


In [132]:
df_matches_clean= df_matches[['journee','lieu_match','date','temp_max', 'temp_min', 'precipitations']]

In [133]:
df_matches_clean['temp_max']=df_matches_clean['temp_max'].map(lambda x : x.replace('°',''))
df_matches_clean['temp_min']=df_matches_clean['temp_min'].map(lambda x : x.replace('°',''))
df_matches_clean['precipitations']=df_matches_clean['precipitations'].map(lambda x : x.replace('mm',''))
df_matches_clean['date']=df_matches_clean['date'].map(lambda x : x.replace('/','-'))

<ipython-input-133-e7c9c720a609>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches_clean['temp_max']=df_matches_clean['temp_max'].map(lambda x : x.replace('°',''))
<ipython-input-133-e7c9c720a609>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches_clean['temp_min']=df_matches_clean['temp_min'].map(lambda x : x.replace('°',''))
<ipython-input-133-e7c9c720a609>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [134]:
df_matches_clean=df_matches_clean.rename(columns={"precipitations":"precipitations_in_mm"})

In [135]:
df_matches_clean

,journee,lieu_match,date,temp_max,temp_min,precipitations_in_mm
0,1,Bordeaux,2020-08-21,24,21,5
1,1,Dijon,2020-08-22,26,18,19
2,1,Lille,2020-08-22,21,17,1
3,1,Monaco,2020-08-23,27,24,1
4,1,Lorient,2020-08-23,19,15,1
...,...,...,...,...,...,...
375,38,Rennes,2021-05-23,15,5,2
376,38,Nantes,2021-05-23,16,8,1
377,38,Saint-Étienne,2021-05-23,17,7,0
378,38,Reims,2021-05-23,14,7,1


In [136]:
df_matches_clean['date_match']=[datetime.strptime(value,"%Y-%m-%d").date() for value in df_matches_clean['date']]

In [137]:
i=0
for value in zip(df_matches_clean['temp_max'],df_matches_clean['temp_min']):
    df_matches_clean.loc[i,'avg_temp']=((int(value[0])+int(value[1]))/2)
    i+=1

In [138]:
# adding values in matchs table
for i,match in df_matches_clean[["date_match","lieu_match","precipitations_in_mm", "avg_temp"]].iterrows() :
    new_match = Matches(match.values[0],match.values[1],match.values[2],match.values[3])
    session.add(new_match)

In [139]:
session.commit()
session.close()

2021-07-01 13:38:37,565 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:38:37,612 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:37,614 INFO sqlalchemy.engine.Engine [generated in 0.00279s] ('2020-08-21', 'Bordeaux', 5.0, 22.5)
2021-07-01 13:38:37,623 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:37,626 INFO sqlalchemy.engine.Engine [cached since 0.01403s ago] ('2020-08-22', 'Dijon', 19.0, 22.0)
2021-07-01 13:38:37,630 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:37,634 INFO sqlalchemy.engine.Engine [cached since 0.0229s ago] ('2020-08-22', 'Lille', 1.0, 19.0)
2021-07-01 13:38:37,639 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:37,641 INFO sqlalchemy.engine.Engine [

2021-07-01 13:38:38,041 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,044 INFO sqlalchemy.engine.Engine [cached since 0.4327s ago] ('2020-09-20', 'Nice', 16.0, 23.0)
2021-07-01 13:38:38,049 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,053 INFO sqlalchemy.engine.Engine [cached since 0.4414s ago] ('2020-09-20', 'Montpellier', 16.0, 23.0)
2021-07-01 13:38:38,056 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,071 INFO sqlalchemy.engine.Engine [cached since 0.4596s ago] ('2020-09-20', 'Brest', 0.0, 17.0)
2021-07-01 13:38:38,073 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,080 INFO sqlalchemy.engine.Engine [cached since 0.4684s ago] ('2020-09-20', 'Metz', 1.0, 19.0)
2021-0

2021-07-01 13:38:38,351 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,354 INFO sqlalchemy.engine.Engine [cached since 0.7423s ago] ('2020-10-18', 'Nantes', 0.0, 11.5)
2021-07-01 13:38:38,357 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,359 INFO sqlalchemy.engine.Engine [cached since 0.7479s ago] ('2020-10-18', 'Angers', 0.0, 12.0)
2021-07-01 13:38:38,362 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,368 INFO sqlalchemy.engine.Engine [cached since 0.7565s ago] ('2020-10-18', 'Saint-Étienne', 0.0, 6.5)
2021-07-01 13:38:38,371 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,374 INFO sqlalchemy.engine.Engine [cached since 0.7627s ago] ('2020-10-18', 'Lille', 2.0, 11.5)
202

2021-07-01 13:38:38,811 INFO sqlalchemy.engine.Engine [cached since 1.2s ago] ('2020-11-08', 'Lyon', 4.0, 13.5)
2021-07-01 13:38:38,816 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,819 INFO sqlalchemy.engine.Engine [cached since 1.208s ago] ('2020-11-20', 'Rennes', 0.0, 7.5)
2021-07-01 13:38:38,822 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,825 INFO sqlalchemy.engine.Engine [cached since 1.214s ago] ('2020-11-20', 'Monaco', 3.0, 14.5)
2021-07-01 13:38:38,829 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38,832 INFO sqlalchemy.engine.Engine [cached since 1.22s ago] ('2020-11-21', 'Brest', 1.0, 10.5)
2021-07-01 13:38:38,841 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:38

2021-07-01 13:38:39,072 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,074 INFO sqlalchemy.engine.Engine [cached since 1.462s ago] ('2020-12-13', 'Nice', 0.0, 11.0)
2021-07-01 13:38:39,077 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,079 INFO sqlalchemy.engine.Engine [cached since 1.468s ago] ('2020-12-13', 'Brest', 10.0, 11.5)
2021-07-01 13:38:39,083 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,087 INFO sqlalchemy.engine.Engine [cached since 1.476s ago] ('2020-12-13', 'Strasbourg', 3.0, 7.0)
2021-07-01 13:38:39,090 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,093 INFO sqlalchemy.engine.Engine [cached since 1.481s ago] ('2020-12-13', 'Nantes', 2.0, 9.5)
2021-07-01 1

...

2021-07-01 13:38:39,839 INFO sqlalchemy.engine.Engine [cached since 2.228s ago] ('2021-02-07', 'Brest', 1.0, 5.5)
2021-07-01 13:38:39,843 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,845 INFO sqlalchemy.engine.Engine [cached since 2.234s ago] ('2021-02-07', 'Nîmes', 1.0, 11.5)
2021-07-01 13:38:39,848 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,850 INFO sqlalchemy.engine.Engine [cached since 2.239s ago] ('2021-02-07', 'Saint-Étienne', 25.0, 7.0)
2021-07-01 13:38:39,855 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:39,857 INFO sqlalchemy.engine.Engine [cached since 2.246s ago] ('2021-02-07', 'Nice', 10.0, 12.5)
2021-07-01 13:38:39,860 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-0

2021-07-01 13:38:40,096 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,098 INFO sqlalchemy.engine.Engine [cached since 2.487s ago] ('2021-02-28', 'Nîmes', 4.0, 11.5)
2021-07-01 13:38:40,104 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,107 INFO sqlalchemy.engine.Engine [cached since 2.495s ago] ('2021-02-28', 'Lille', 0.0, 8.0)
2021-07-01 13:38:40,109 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,111 INFO sqlalchemy.engine.Engine [cached since 2.5s ago] ('2021-02-28', 'Marseille', 4.0, 13.0)
2021-07-01 13:38:40,114 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,125 INFO sqlalchemy.engine.Engine [cached since 2.514s ago] ('2021-03-03', 'Lyon', 0.0, 12.0)
2021-07-01 13:38

2021-07-01 13:38:40,422 INFO sqlalchemy.engine.Engine [cached since 2.81s ago] ('2021-04-03', 'Monaco', 7.0, 14.0)
2021-07-01 13:38:40,426 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,429 INFO sqlalchemy.engine.Engine [cached since 2.817s ago] ('2021-04-03', 'Paris', 0.0, 8.5)
2021-07-01 13:38:40,431 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,438 INFO sqlalchemy.engine.Engine [cached since 2.827s ago] ('2021-04-03', 'Lens', 0.0, 7.5)
2021-07-01 13:38:40,444 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,447 INFO sqlalchemy.engine.Engine [cached since 2.836s ago] ('2021-04-04', 'Angers', 0.0, 8.0)
2021-07-01 13:38:40,451 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40

2021-07-01 13:38:40,826 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,829 INFO sqlalchemy.engine.Engine [cached since 3.217s ago] ('2021-04-25', 'Lens', 0.0, 8.5)
2021-07-01 13:38:40,832 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,835 INFO sqlalchemy.engine.Engine [cached since 3.223s ago] ('2021-04-25', 'Rennes', 0.0, 12.0)
2021-07-01 13:38:40,838 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,840 INFO sqlalchemy.engine.Engine [cached since 3.229s ago] ('2021-04-25', 'Strasbourg', 0.0, 12.0)
2021-07-01 13:38:40,843 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:40,846 INFO sqlalchemy.engine.Engine [cached since 3.234s ago] ('2021-04-25', 'Lorient', 0.0, 13.5)
2021-07-01

2021-07-01 13:38:41,147 INFO sqlalchemy.engine.Engine [cached since 3.536s ago] ('2021-05-16', 'Nice', 0.0, 16.5)
2021-07-01 13:38:41,150 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:41,153 INFO sqlalchemy.engine.Engine [cached since 3.541s ago] ('2021-05-16', 'Lorient', 7.0, 12.0)
2021-07-01 13:38:41,156 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:41,161 INFO sqlalchemy.engine.Engine [cached since 3.549s ago] ('2021-05-16', 'Dijon', 7.0, 11.5)
2021-07-01 13:38:41,164 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:38:41,169 INFO sqlalchemy.engine.Engine [cached since 3.558s ago] ('2021-05-23', 'Angers', 1.0, 11.5)
2021-07-01 13:38:41,172 INFO sqlalchemy.engine.Engine INSERT INTO matches (date, place, rainfall, temperature) VALUES (?, ?, ?, ?)
2021-07-01 13:

In [182]:
matchs_from_sql_table=pd.read_sql_table('teams', 'sqlite:///football_orm.sqlite') 

In [183]:
matchs_from_sql_table

,id,name,city,coach_name
0,1,Angers Sporting Club de l'Ouest,Angers,Gérald BATICLE
1,2,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET
2,3,Stade Brestois 29,Brest,Michel DER ZAKARIAN
3,4,Dijon Football Côte-d'Or,Dijon,David LINARES
4,5,Racing Club de Lens,Lens,Franck HAISE
5,6,LOSC Lille,Lille,Christophe GALTIER
6,7,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER
7,8,Olympique Lyonnais,Lyon,Peter BOSZ
8,9,Olympique de Marseille,Marseille,Jorge SAMPAOLI
9,10,Football Club de Metz,Metz,Frédéric ANTONETTI


### Matchs & teams table

In [141]:
df_goals_dom=pd.read_csv('goals_domGT.csv')
df_goals_ext=pd.read_csv('goal_extGT.csv')

In [142]:
df_goals_dom['date']=[value.split('T')[0] for value in df_goals_dom['date']]
df_goals_ext['date']=[value.split('T')[0] for value in df_goals_ext['date']]

In [143]:
df_goals_dom['date']=[datetime.strptime(value,"%Y-%m-%d").date() for value in df_goals_dom['date']]
df_goals_ext['date']=[datetime.strptime(value,"%Y-%m-%d").date() for value in df_goals_ext['date']]

In [144]:
indice_match=[]
for index, row in df_goals_dom[['lieu_match','date']].iterrows():
    #thistuple=(row[0], row[1])
    indice_match.append((row[0], row[1]))
df_goals_dom['indice_match']=indice_match

In [145]:
indice_match=[]
for index, row in df_goals_ext[['lieu_match','date']].iterrows():
    #thistuple=(row[0], row[1])
    indice_match.append((row[0], row[1]))
df_goals_ext['indice_match']=indice_match

In [146]:
result = session.query(Matches).all()

2021-07-01 13:38:42,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:38:42,510 INFO sqlalchemy.engine.Engine SELECT matches.id AS matches_id, matches.date AS matches_date, matches.place AS matches_place, matches.rainfall AS matches_rainfall, matches.temperature AS matches_temperature 
FROM matches
2021-07-01 13:38:42,512 INFO sqlalchemy.engine.Engine [generated in 0.00247s] ()


In [147]:
df_goals_ext_match_id=[]
df_goals_dom_match_id=[]
for match_obj, value_ext,value_dom in zip(result, df_goals_ext['indice_match'],df_goals_dom['indice_match']):
    if (match_obj.place,match_obj.date) == value_ext:
        df_goals_ext_match_id.append(match_obj.id)
    if (match_obj.place,match_obj.date) == value_dom:
        df_goals_dom_match_id.append(match_obj.id)    
df_goals_ext['match_id']=df_goals_ext_match_id
df_goals_dom['match_id']= df_goals_dom_match_id

In [148]:
for index,value in enumerate(df_goals_dom['vainqueur']) : 
    if value == 'ex-aequo':
        df_goals_dom.loc[index,'points']= 1
    elif value == 'domicile' :
        df_goals_dom.loc[index,'points']= 3
    else :
        df_goals_dom.loc[index,'points']= 0

In [149]:
for index,value in enumerate(df_goals_ext['vainqueur']) : 
    if value == 'ex-aequo':
        df_goals_ext.loc[index,'points']= 1
    elif value == 'exterieur' :
        df_goals_ext.loc[index,'points']= 3
    else :
        df_goals_ext.loc[index,'points']= 0
df_goals_ext

,Exterior_team,score_exterior,scorer_names_ext,lieu_match,date,journee,vainqueur,indice_match,match_id,points
0,Nantes,0,[],Bordeaux,2020-08-21,1,ex-aequo,"(Bordeaux, 2020-08-21)",1,1.0
1,Angers,1,['I. Traoré'],Dijon,2020-08-22,1,exterieur,"(Dijon, 2020-08-22)",2,3.0
2,Rennes,1,['D. Da Silva'],Lille,2020-08-22,1,ex-aequo,"(Lille, 2020-08-22)",3,1.0
3,Reims,2,"['B. Dia', 'E. Touré']",Monaco,2020-08-23,1,ex-aequo,"(Monaco, 2020-08-23)",4,1.0
4,Strasbourg,1,['M. Chahiri'],Lorient,2020-08-23,1,domicile,"(Lorient, 2020-08-23)",5,0.0
...,...,...,...,...,...,...,...,...,...,...
375,Nîmes,0,[],Rennes,2021-05-23,38,domicile,"(Rennes, 2021-05-23)",376,0.0
376,Montpellier,2,"['G. Laborde', 'A. Delort']",Nantes,2021-05-23,38,exterieur,"(Nantes, 2021-05-23)",377,3.0
377,Dijon,1,['A. Kamara'],Saint-Étienne,2021-05-23,38,exterieur,"(Saint-Étienne, 2021-05-23)",378,3.0
378,Bordeaux,2,"['Y. Adli', 'E. Kwateng']",Reims,2021-05-23,38,exterieur,"(Reims, 2021-05-23)",379,3.0


In [150]:
i=0
for value in df_goals_ext['Exterior_team']:
    if value in teams_id.keys():
        df_goals_ext.loc[i,'team_id']=teams_id[value]
    i+=1
df_goals_ext

,Exterior_team,score_exterior,scorer_names_ext,lieu_match,date,journee,vainqueur,indice_match,match_id,points,team_id
0,Nantes,0,[],Bordeaux,2020-08-21,1,ex-aequo,"(Bordeaux, 2020-08-21)",1,1.0,13.0
1,Angers,1,['I. Traoré'],Dijon,2020-08-22,1,exterieur,"(Dijon, 2020-08-22)",2,3.0,1.0
2,Rennes,1,['D. Da Silva'],Lille,2020-08-22,1,ex-aequo,"(Lille, 2020-08-22)",3,1.0,18.0
3,Reims,2,"['B. Dia', 'E. Touré']",Monaco,2020-08-23,1,ex-aequo,"(Monaco, 2020-08-23)",4,1.0,17.0
4,Strasbourg,1,['M. Chahiri'],Lorient,2020-08-23,1,domicile,"(Lorient, 2020-08-23)",5,0.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...
375,Nîmes,0,[],Rennes,2021-05-23,38,domicile,"(Rennes, 2021-05-23)",376,0.0,15.0
376,Montpellier,2,"['G. Laborde', 'A. Delort']",Nantes,2021-05-23,38,exterieur,"(Nantes, 2021-05-23)",377,3.0,12.0
377,Dijon,1,['A. Kamara'],Saint-Étienne,2021-05-23,38,exterieur,"(Saint-Étienne, 2021-05-23)",378,3.0,4.0
378,Bordeaux,2,"['Y. Adli', 'E. Kwateng']",Reims,2021-05-23,38,exterieur,"(Reims, 2021-05-23)",379,3.0,2.0


In [151]:
i=0
for value in df_goals_dom['Domicile_team']:
    if value in teams_id.keys():
        df_goals_dom.loc[i,'team_id']=teams_id[value]
    i+=1
df_goals_dom

,Domicile_team,score_dom,scorer_names_dom,lieu_match,date,journee,vainqueur,indice_match,match_id,points,team_id
0,Bordeaux,0,[],Bordeaux,2020-08-21,1,ex-aequo,"(Bordeaux, 2020-08-21)",1,1.0,2.0
1,Dijon,0,[],Dijon,2020-08-22,1,exterieur,"(Dijon, 2020-08-22)",2,0.0,4.0
2,Lille,1,['J. Bamba'],Lille,2020-08-22,1,ex-aequo,"(Lille, 2020-08-22)",3,1.0,6.0
3,Monaco,2,"['A. Disasi', 'B. Badiashile']",Monaco,2020-08-23,1,ex-aequo,"(Monaco, 2020-08-23)",4,1.0,11.0
4,Lorient,3,"['Y. Wissa', 'A. Grbic(s.p.)', 'P. Hamel']",Lorient,2020-08-23,1,domicile,"(Lorient, 2020-08-23)",5,3.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...
375,Rennes,2,"['S. Guirassy', 'B. Bourigeaud']",Rennes,2021-05-23,38,domicile,"(Rennes, 2021-05-23)",376,3.0,18.0
376,Nantes,1,['R. Kolo Muani'],Nantes,2021-05-23,38,exterieur,"(Nantes, 2021-05-23)",377,0.0,13.0
377,Saint-Étienne,0,[],Saint-Étienne,2021-05-23,38,exterieur,"(Saint-Étienne, 2021-05-23)",378,0.0,19.0
378,Reims,1,['E. Touré'],Reims,2021-05-23,38,exterieur,"(Reims, 2021-05-23)",379,0.0,17.0


In [152]:
df_goals_dom['home']=1
df_goals_ext['home']=0

In [153]:
df_goals_dom=df_goals_dom.rename(columns={"score_dom":"goals"})
df_goals_ext=df_goals_ext.rename(columns={"score_exterior":"goals"})

In [154]:
df_matchs_teams= pd.concat([df_goals_dom[['team_id','match_id','home','points','goals']],df_goals_ext[['team_id','match_id','home','points','goals']]])

In [155]:
# adding values in matchs_teams table
for i,match in df_matchs_teams.iterrows() :
    new_match = MatchsTeamsLink(match.values[0],match.values[1],match.values[2],match.values[3],match.values[4])
    session.add(new_match)
session.commit()
session.close()

2021-07-01 13:38:49,211 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,222 INFO sqlalchemy.engine.Engine [generated in 0.01161s] (2.0, 1.0, 1, 1.0, 0.0)
2021-07-01 13:38:49,228 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,230 INFO sqlalchemy.engine.Engine [cached since 0.01925s ago] (4.0, 2.0, 1, 0.0, 0.0)
2021-07-01 13:38:49,233 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,235 INFO sqlalchemy.engine.Engine [cached since 0.02477s ago] (6.0, 3.0, 1, 1.0, 1.0)
2021-07-01 13:38:49,263 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,265 INFO sqlalchemy.engine.Engine [cached since 0.05391s ago] (11.

2021-07-01 13:38:49,504 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,506 INFO sqlalchemy.engine.Engine [cached since 0.2957s ago] (18.0, 33.0, 1, 3.0, 2.0)
2021-07-01 13:38:49,509 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,511 INFO sqlalchemy.engine.Engine [cached since 0.3007s ago] (14.0, 34.0, 1, 0.0, 0.0)
2021-07-01 13:38:49,515 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,518 INFO sqlalchemy.engine.Engine [cached since 0.3074s ago] (12.0, 35.0, 1, 3.0, 4.0)
2021-07-01 13:38:49,521 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,523 INFO sqlalchemy.engine.Engine [cached since 0.3127s ago

2021-07-01 13:38:49,773 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,775 INFO sqlalchemy.engine.Engine [cached since 0.5642s ago] (20.0, 65.0, 1, 0.0, 2.0)
2021-07-01 13:38:49,778 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,781 INFO sqlalchemy.engine.Engine [cached since 0.5699s ago] (11.0, 66.0, 1, 1.0, 1.0)
2021-07-01 13:38:49,784 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,786 INFO sqlalchemy.engine.Engine [cached since 0.5756s ago] (13.0, 67.0, 1, 3.0, 3.0)
2021-07-01 13:38:49,793 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:49,796 INFO sqlalchemy.engine.Engine [cached since 0.5855s ago

2021-07-01 13:38:50,091 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,093 INFO sqlalchemy.engine.Engine [cached since 0.8823s ago] (15.0, 97.0, 1, 0.0, 1.0)
2021-07-01 13:38:50,097 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,099 INFO sqlalchemy.engine.Engine [cached since 0.8879s ago] (7.0, 98.0, 1, 0.0, 0.0)
2021-07-01 13:38:50,104 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,151 INFO sqlalchemy.engine.Engine [cached since 0.9403s ago] (14.0, 99.0, 1, 0.0, 1.0)
2021-07-01 13:38:50,154 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,156 INFO sqlalchemy.engine.Engine [cached since 0.9455s ago]

2021-07-01 13:38:50,510 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,513 INFO sqlalchemy.engine.Engine [cached since 1.302s ago] (17.0, 129.0, 1, 1.0, 0.0)
2021-07-01 13:38:50,516 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,520 INFO sqlalchemy.engine.Engine [cached since 1.309s ago] (10.0, 130.0, 1, 0.0, 1.0)
2021-07-01 13:38:50,524 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,542 INFO sqlalchemy.engine.Engine [cached since 1.331s ago] (19.0, 131.0, 1, 1.0, 0.0)
2021-07-01 13:38:50,546 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:50,548 INFO sqlalchemy.engine.Engine [cached since 1.337s ago]

...

2021-07-01 13:38:55,706 INFO sqlalchemy.engine.Engine [cached since 6.496s ago] (13.0, 230.0, 0, 1.0, 1.0)
2021-07-01 13:38:55,711 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:55,713 INFO sqlalchemy.engine.Engine [cached since 6.502s ago] (17.0, 231.0, 0, 0.0, 0.0)
2021-07-01 13:38:55,740 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:55,743 INFO sqlalchemy.engine.Engine [cached since 6.532s ago] (20.0, 232.0, 0, 0.0, 0.0)
2021-07-01 13:38:55,746 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:55,754 INFO sqlalchemy.engine.Engine [cached since 6.544s ago] (18.0, 233.0, 0, 1.0, 0.0)
2021-07-01 13:38:55,758 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_go

2021-07-01 13:38:56,006 INFO sqlalchemy.engine.Engine [cached since 6.795s ago] (10.0, 262.0, 0, 3.0, 2.0)
2021-07-01 13:38:56,010 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,013 INFO sqlalchemy.engine.Engine [cached since 6.802s ago] (16.0, 263.0, 0, 3.0, 4.0)
2021-07-01 13:38:56,016 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,020 INFO sqlalchemy.engine.Engine [cached since 6.809s ago] (3.0, 264.0, 0, 0.0, 0.0)
2021-07-01 13:38:56,023 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,026 INFO sqlalchemy.engine.Engine [cached since 6.816s ago] (5.0, 265.0, 0, 1.0, 2.0)
2021-07-01 13:38:56,030 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goal

2021-07-01 13:38:56,319 INFO sqlalchemy.engine.Engine [cached since 7.108s ago] (5.0, 294.0, 0, 3.0, 2.0)
2021-07-01 13:38:56,323 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,325 INFO sqlalchemy.engine.Engine [cached since 7.115s ago] (2.0, 295.0, 0, 0.0, 1.0)
2021-07-01 13:38:56,330 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,374 INFO sqlalchemy.engine.Engine [cached since 7.164s ago] (1.0, 296.0, 0, 1.0, 0.0)
2021-07-01 13:38:56,378 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,380 INFO sqlalchemy.engine.Engine [cached since 7.17s ago] (17.0, 297.0, 0, 3.0, 1.0)
2021-07-01 13:38:56,385 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals)

2021-07-01 13:38:56,702 INFO sqlalchemy.engine.Engine [cached since 7.492s ago] (14.0, 326.0, 0, 0.0, 0.0)
2021-07-01 13:38:56,705 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,707 INFO sqlalchemy.engine.Engine [cached since 7.497s ago] (10.0, 327.0, 0, 1.0, 0.0)
2021-07-01 13:38:56,710 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,728 INFO sqlalchemy.engine.Engine [cached since 7.518s ago] (20.0, 328.0, 0, 1.0, 1.0)
2021-07-01 13:38:56,731 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:56,739 INFO sqlalchemy.engine.Engine [cached since 7.528s ago] (11.0, 329.0, 0, 3.0, 3.0)
2021-07-01 13:38:56,746 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_go

2021-07-01 13:38:57,024 INFO sqlalchemy.engine.Engine [cached since 7.813s ago] (4.0, 358.0, 0, 0.0, 0.0)
2021-07-01 13:38:57,027 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:57,030 INFO sqlalchemy.engine.Engine [cached since 7.82s ago] (11.0, 359.0, 0, 3.0, 1.0)
2021-07-01 13:38:57,037 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:57,039 INFO sqlalchemy.engine.Engine [cached since 7.829s ago] (16.0, 360.0, 0, 1.0, 1.0)
2021-07-01 13:38:57,043 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goals) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:38:57,046 INFO sqlalchemy.engine.Engine [cached since 7.835s ago] (19.0, 361.0, 0, 1.0, 0.0)
2021-07-01 13:38:57,051 INFO sqlalchemy.engine.Engine INSERT INTO matches_teams (team_id, matchs_id, home, points, team_goal

In [156]:
pd.read_sql_table('matches_teams', 'sqlite:///football_orm.sqlite') 

,id,team_id,matchs_id,home,points,team_goals
0,1,2,1,True,1,0
1,2,4,2,True,0,0
2,3,6,3,True,1,1
3,4,11,4,True,1,2
4,5,7,5,True,3,3
...,...,...,...,...,...,...
755,756,15,376,False,0,0
756,757,12,377,False,3,2
757,758,4,378,False,3,1
758,759,2,379,False,3,2


### Goals table

#### Recuperation des buteurs

In [157]:
scorers_list=[]
matchs_list=[]
for scorers,matchs in zip(df_goals_ext['scorer_names_ext'],df_goals_ext['match_id']):
    scorers=scorers.replace("'","").strip('][').split(', ')
    for scorer in scorers :
        if scorer != '' : 
            scorers_list.append(scorer)
            matchs_list.append(matchs)

In [158]:
for scorers,matchs in zip(df_goals_dom['scorer_names_dom'],df_goals_dom['match_id']):
    scorers=scorers.replace("'","").strip('][').split(', ')
    for scorer in scorers :
        if scorer != '' : 
            scorers_list.append(scorer)
            matchs_list.append(matchs)
    
dict_goals= {'scorer':scorers_list, 'match_id':matchs_list}
df_goals = pd.DataFrame(data=dict_goals)   

In [159]:
for i, scorer in enumerate(df_goals['scorer']):
    if len(scorer.split('(')) == 1:
        df_goals.loc[i,'scorer'] = scorer.split('(')[0]
        df_goals.loc[i,'goal_type']= 'normal'
    else :
        df_goals.loc[i,'scorer'] = scorer.split('(')[0]
        df_goals.loc[i, 'goal_type']= scorer.split('(')[1].replace(')','')
    
    

In [160]:
df_goals[df_goals['goal_type'] == 's.p.']

,scorer,match_id,goal_type
5,G. Kakuta,7,s.p.
6,M. Depay,8,s.p.
11,K. Dolberg,13,s.p.
26,G. Kakuta,26,s.p.
35,K. Mbappé,34,s.p.
...,...,...,...
1024,Neymar,362,s.p.
1034,A. Milik,365,s.p.
1035,U. Hwang,366,s.p.
1038,Y. Wissa,369,s.p.


#### Ajout des buteurs id

In [161]:
## recuperation des id 
players_id = {}
result = session.query(Players).all()
for player_obj in result:
    players_id[player_obj.name] = player_obj.id

2021-07-01 13:39:03,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:39:03,572 INFO sqlalchemy.engine.Engine SELECT players.id AS players_id, players.name AS players_name, players.position AS players_position, players.birthdate AS players_birthdate, players.nationality AS players_nationality, players.team_id AS players_team_id 
FROM players
2021-07-01 13:39:03,574 INFO sqlalchemy.engine.Engine [generated in 0.00286s] ()


In [162]:
##ajout des id 
# 0 when player not found in player table
i=0
for value in df_goals['scorer']:
    if value in players_id.keys():
        df_goals.loc[i,'player_id']=players_id[value]
    else: 
        df_goals.loc[i,'player_id']='0'
    i+=1

In [163]:
joueurs_transfered=set(df_goals.query("player_id == '0'")['scorer'])

In [164]:
joueurs_transfered

{'A. Scheidler',
 'D. Pereira Da Costa',
 'J. Maja',
 'K. Denkey',
 'K. Lala',
 'M. Dembélé',
 'M. Niang',
 'M. Sanson',
 'N. Radonjic',
 'Pablo',
 'Raphinha',
 'Y. Sabaly'}

#### Update of players table with missing players information**

In [165]:
pd.read_sql_table('teams', 'sqlite:///football_orm.sqlite') 

,id,name,city,coach_name
0,1,Angers Sporting Club de l'Ouest,Angers,Gérald BATICLE
1,2,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET
2,3,Stade Brestois 29,Brest,Michel DER ZAKARIAN
3,4,Dijon Football Côte-d'Or,Dijon,David LINARES
4,5,Racing Club de Lens,Lens,Franck HAISE
5,6,LOSC Lille,Lille,Christophe GALTIER
6,7,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER
7,8,Olympique Lyonnais,Lyon,Peter BOSZ
8,9,Olympique de Marseille,Marseille,Jorge SAMPAOLI
9,10,Football Club de Metz,Metz,Frédéric ANTONETTI


In [166]:
Scheidler= Players("A. Scheidler", "Att.", datetime.strptime("4/01/98", '%d/%m/%y'), "FRA", 4)
Pereira_Da_Costa = Players("D. Pereira Da Costa", "Mil.",datetime.strptime("5/01/01", '%d/%m/%y'), "POR",5)
Maja = Players("J. Maja", "Att.", datetime.strptime("27/12/98", '%d/%m/%y'), "GBR", 2)
Denkey = Players("K. Denkey","Att.", datetime.strptime("30/11/00", '%d/%m/%y'), "TOG",  15)
Lala = Players("K. Lala","Déf.", datetime.strptime("3/10/91", '%d/%m/%y'),'FRA', 20)
Dembélé = Players("M. Dembélé","Att.",datetime.strptime("12/01/96", '%d/%m/%y'), "FRA", 8)
Niang = Players("M. Niang","Att.", datetime.strptime("19/12/94", '%d/%m/%y'), "SEN",  18)
Sanson = Players("M. Sanson", "Mil.", datetime.strptime("18/08/94", '%d/%m/%y'), "FRA",  9)
Radonjic = Players("N. Radonjic",  "Att.", datetime.strptime("15/02/94", '%d/%m/%y'), "SRB",9)
Pablo= Players("Pablo", "Déf", datetime.strptime("21/06/91", '%d/%m/%y'), "BRA", 2)
Raphinha = Players("Raphinha","Att.",  datetime.strptime("14/12/96", '%d/%m/%y'),"BRA", 18)
Sabaly = Players("Y. Sabaly", "Déf", datetime.strptime("5/03/93", '%d/%m/%y'),"SEN", 2)


In [167]:
session.add(Scheidler)
session.add(Pereira_Da_Costa)
session.add(Maja )
session.add(Denkey )
session.add(Lala )
session.add(Dembélé )
session.add(Niang )
session.add(Sanson )
session.add(Radonjic )
session.add(Pablo)
session.add(Raphinha )
session.add(Sabaly)

In [168]:
session.commit()
session.close()

2021-07-01 13:39:06,599 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:39:06,603 INFO sqlalchemy.engine.Engine [cached since 37.13s ago] ('A. Scheidler', 'Att.', '1998-01-04', 'FRA', 4)
2021-07-01 13:39:06,609 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:39:06,611 INFO sqlalchemy.engine.Engine [cached since 37.14s ago] ('D. Pereira Da Costa', 'Mil.', '2001-01-05', 'POR', 5)
2021-07-01 13:39:06,615 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:39:06,618 INFO sqlalchemy.engine.Engine [cached since 37.15s ago] ('J. Maja', 'Att.', '1998-12-27', 'GBR', 2)
2021-07-01 13:39:06,622 INFO sqlalchemy.engine.Engine INSERT INTO players (name, position, birthdate, nationality, team_id) VALUES (?, ?, ?, ?, ?)
2021-07-01 13:39:06,623

In [169]:
pd.read_sql_table('players', 'sqlite:///football_orm.sqlite') 

,id,name,position,birthdate,nationality,team_id
0,1,P. Bernardoni,Gar.,1997-04-18,FRA,1
1,2,L. Butelle,Gar.,1983-04-03,FRA,1
2,3,D. Petkovic,Gar.,1993-05-25,MNE,1
3,4,A. Bamba,Déf.,1990-04-25,CIV,1
4,5,K. Boma,Déf.,2002-11-20,FRA,1
...,...,...,...,...,...,...
631,632,M. Sanson,Mil.,1994-08-18,FRA,9
632,633,N. Radonjic,Att.,1994-02-15,SRB,9
633,634,Pablo,Déf,1991-06-21,BRA,2
634,635,Raphinha,Att.,1996-12-14,BRA,18


#### Update des players id dans goal table

In [170]:
## recuperation des id 
players_id = {}
result = session.query(Players).all()
for player_obj in result:
    players_id[player_obj.name] = player_obj.id

2021-07-01 13:39:07,144 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-01 13:39:07,149 INFO sqlalchemy.engine.Engine SELECT players.id AS players_id, players.name AS players_name, players.position AS players_position, players.birthdate AS players_birthdate, players.nationality AS players_nationality, players.team_id AS players_team_id 
FROM players
2021-07-01 13:39:07,151 INFO sqlalchemy.engine.Engine [cached since 3.58s ago] ()


In [171]:
##ajout des id 
# 0 when player not found in player table
i=0
for value in df_goals['scorer']:
    if value in players_id.keys():
        df_goals.loc[i,'player_id']=players_id[value]
    else: 
        df_goals.loc[i,'player_id']='0'
    i+=1

In [172]:
joueurs_transfered=set(df_goals.query("player_id == '0'")['scorer'])

In [173]:
joueurs_transfered

set()

In [174]:
# adding values in matchs_teams table
for i,goal in df_goals.iterrows() :
    new_goal = Goals(goal.values[3],goal.values[1],goal.values[2])
    session.add(new_goal)
session.commit()
session.close()

2021-07-01 13:39:10,321 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,323 INFO sqlalchemy.engine.Engine [generated in 0.00256s] (12, 2, 'normal')
2021-07-01 13:39:10,331 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,334 INFO sqlalchemy.engine.Engine [cached since 0.01342s ago] (530, 3, 'normal')
2021-07-01 13:39:10,338 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,340 INFO sqlalchemy.engine.Engine [cached since 0.01966s ago] (520, 4, 'normal')
2021-07-01 13:39:10,343 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,347 INFO sqlalchemy.engine.Engine [cached since 0.0261s ago] (525, 4, 'normal')
2021-07-01 13:39:10,360 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?

2021-07-01 13:39:10,611 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,614 INFO sqlalchemy.engine.Engine [cached since 0.2935s ago] (473, 34, 'normal')
2021-07-01 13:39:10,619 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,622 INFO sqlalchemy.engine.Engine [cached since 0.3012s ago] (27, 35, 'normal')
2021-07-01 13:39:10,625 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,627 INFO sqlalchemy.engine.Engine [cached since 0.3065s ago] (76, 36, 'csc')
2021-07-01 13:39:10,630 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,632 INFO sqlalchemy.engine.Engine [cached since 0.3114s ago] (212, 36, 'normal')
2021-07-01 13:39:10,636 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (

2021-07-01 13:39:10,947 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,950 INFO sqlalchemy.engine.Engine [cached since 0.6298s ago] (216, 63, 'normal')
2021-07-01 13:39:10,953 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,955 INFO sqlalchemy.engine.Engine [cached since 0.6346s ago] (218, 63, 's.p.')
2021-07-01 13:39:10,958 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,960 INFO sqlalchemy.engine.Engine [cached since 0.6397s ago] (217, 63, 'normal')
2021-07-01 13:39:10,963 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:10,966 INFO sqlalchemy.engine.Engine [cached since 0.6454s ago] (627, 64, 'normal')
2021-07-01 13:39:10,970 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUE

2021-07-01 13:39:11,236 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,238 INFO sqlalchemy.engine.Engine [cached since 0.9184s ago] (152, 90, 'normal')
2021-07-01 13:39:11,241 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,243 INFO sqlalchemy.engine.Engine [cached since 0.9231s ago] (632, 91, 'normal')
2021-07-01 13:39:11,246 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,248 INFO sqlalchemy.engine.Engine [cached since 0.9278s ago] (349, 92, 'normal')
2021-07-01 13:39:11,251 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,254 INFO sqlalchemy.engine.Engine [cached since 0.9342s ago] (355, 92, 'normal')
2021-07-01 13:39:11,257 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VAL

2021-07-01 13:39:11,522 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,530 INFO sqlalchemy.engine.Engine [cached since 1.21s ago] (49, 113, 'normal')
2021-07-01 13:39:11,534 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,541 INFO sqlalchemy.engine.Engine [cached since 1.221s ago] (24, 115, 'normal')
2021-07-01 13:39:11,545 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,552 INFO sqlalchemy.engine.Engine [cached since 1.232s ago] (27, 115, 'normal')
2021-07-01 13:39:11,555 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:11,557 INFO sqlalchemy.engine.Engine [cached since 1.237s ago] (18, 115, 'normal')
2021-07-01 13:39:11,562 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (

...

2021-07-01 13:39:17,142 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,145 INFO sqlalchemy.engine.Engine [cached since 6.825s ago] (231, 271, 'normal')
2021-07-01 13:39:17,148 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,151 INFO sqlalchemy.engine.Engine [cached since 6.831s ago] (563, 273, 'normal')
2021-07-01 13:39:17,154 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,158 INFO sqlalchemy.engine.Engine [cached since 6.838s ago] (584, 273, 's.p.')
2021-07-01 13:39:17,162 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,169 INFO sqlalchemy.engine.Engine [cached since 6.849s ago] (418, 274, 'normal')
2021-07-01 13:39:17,173 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUE

2021-07-01 13:39:17,389 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,391 INFO sqlalchemy.engine.Engine [cached since 7.07s ago] (331, 301, 'normal')
2021-07-01 13:39:17,393 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,395 INFO sqlalchemy.engine.Engine [cached since 7.074s ago] (325, 301, 'normal')
2021-07-01 13:39:17,398 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,401 INFO sqlalchemy.engine.Engine [cached since 7.081s ago] (325, 301, 's.p.')
2021-07-01 13:39:17,403 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,406 INFO sqlalchemy.engine.Engine [cached since 7.086s ago] (132, 303, 'normal')
2021-07-01 13:39:17,408 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES

2021-07-01 13:39:17,608 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,610 INFO sqlalchemy.engine.Engine [cached since 7.29s ago] (492, 324, 'normal')
2021-07-01 13:39:17,613 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,615 INFO sqlalchemy.engine.Engine [cached since 7.294s ago] (492, 324, 's.p.')
2021-07-01 13:39:17,619 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,621 INFO sqlalchemy.engine.Engine [cached since 7.301s ago] (490, 324, 'normal')
2021-07-01 13:39:17,624 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,626 INFO sqlalchemy.engine.Engine [cached since 7.306s ago] (142, 325, 'csc')
2021-07-01 13:39:17,629 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?

2021-07-01 13:39:17,882 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,885 INFO sqlalchemy.engine.Engine [cached since 7.564s ago] (218, 347, 's.p.')
2021-07-01 13:39:17,889 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,891 INFO sqlalchemy.engine.Engine [cached since 7.57s ago] (210, 347, 'normal')
2021-07-01 13:39:17,894 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,896 INFO sqlalchemy.engine.Engine [cached since 7.575s ago] (83, 348, 'normal')
2021-07-01 13:39:17,898 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:17,901 INFO sqlalchemy.engine.Engine [cached since 7.58s ago] (355, 349, 'normal')
2021-07-01 13:39:17,904 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (

2021-07-01 13:39:18,111 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:18,113 INFO sqlalchemy.engine.Engine [cached since 7.793s ago] (218, 369, 's.p.')
2021-07-01 13:39:18,116 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:18,118 INFO sqlalchemy.engine.Engine [cached since 7.798s ago] (195, 369, 'normal')
2021-07-01 13:39:18,121 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:18,123 INFO sqlalchemy.engine.Engine [cached since 7.802s ago] (21, 371, 'normal')
2021-07-01 13:39:18,125 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES (?, ?, ?)
2021-07-01 13:39:18,131 INFO sqlalchemy.engine.Engine [cached since 7.81s ago] (244, 374, 'normal')
2021-07-01 13:39:18,134 INFO sqlalchemy.engine.Engine INSERT INTO goals (player_id, match_id, goal_type) VALUES 

In [175]:
pd.read_sql_table('goals', 'sqlite:///football_orm.sqlite') 

,id,player_id,match_id,goal_type
0,1,12,2,normal
1,2,530,3,normal
2,3,520,4,normal
3,4,525,4,normal
4,5,610,5,normal
...,...,...,...,...
1044,1045,550,376,normal
1045,1046,539,376,normal
1046,1047,386,377,normal
1047,1048,525,379,normal


NameError: name 'metadata' is not defined